In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10

In [15]:
tf.config.list_physical_devices('GPU')

[]

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)
print(y_train.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
(50000, 32, 32, 3)
(50000, 1)


In [4]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [24]:
def my_model():
    inputs = keras.Input((32, 32, 3), name='input')
    x = layers.Conv2D(32, 3,
                      name='conv2d',
                     padding='same',
                     kernel_regularizer=regularizers.l2(0.01))(inputs)
    x = layers.BatchNormalization(name='batch_norm1')(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='maxpool')(x) #(2,2) is default
    x = layers.Conv2D(64, 3,
                      name='conv2d_2',
                     padding='same',
                     kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization(name='batch_norm2')(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D(name='maxpool_2')(x)
    x = layers.Conv2D(128, 3,
                      name='conv2d_3',
                     padding='same',
                     kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization(name='batch_norm3')(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64,
                     activation='relu',
                     name='dense',
                    kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5, name='dropout')(x)
    outputs = layers.Dense(10, name='output')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

model = my_model()

model.compile(

    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=['accuracy']
)

model.summary()

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                   │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_norm1 (BatchNormalization)     │ (None, 32, 32, 32)          │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_24 (ReLU)                      │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ maxpool (MaxPooling2D)               │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_norm2 (BatchNormalization)     │ (None, 16, 16, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_25 (ReLU)                      │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ maxpool_2 (MaxPooling2D)             │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_norm3 (BatchNormalization)     │ (None, 8, 8, 128)           │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_26 (ReLU)                      │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_10 (Flatten)                 │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         524,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 619,146 (2.36 MB)

 Trainable params: 618,698 (2.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [25]:
model.fit(x_train, y_train, batch_size=64, epochs=150, verbose=2)

Epoch 1/150
782/782 - 11s - 13ms/step - accuracy: 0.3532 - loss: 3.0311
Epoch 2/150
782/782 - 9s - 11ms/step - accuracy: 0.4699 - loss: 1.9111
Epoch 3/150
782/782 - 9s - 11ms/step - accuracy: 0.5106 - loss: 1.6348
Epoch 4/150
782/782 - 9s - 11ms/step - accuracy: 0.5310 - loss: 1.5236
Epoch 5/150
782/782 - 9s - 11ms/step - accuracy: 0.5534 - loss: 1.4524
Epoch 6/150
782/782 - 9s - 11ms/step - accuracy: 0.5641 - loss: 1.4111
Epoch 7/150
782/782 - 9s - 11ms/step - accuracy: 0.5718 - loss: 1.3785
Epoch 8/150
782/782 - 9s - 11ms/step - accuracy: 0.5864 - loss: 1.3504
Epoch 9/150
782/782 - 9s - 11ms/step - accuracy: 0.5947 - loss: 1.3317
Epoch 10/150
782/782 - 9s - 11ms/step - accuracy: 0.6032 - loss: 1.3052
Epoch 11/150
782/782 - 9s - 11ms/step - accuracy: 0.6105 - loss: 1.2922
Epoch 12/150
782/782 - 9s - 11ms/step - accuracy: 0.6161 - loss: 1.2795
Epoch 13/150
782/782 - 9s - 11ms/step - accuracy: 0.6226 - loss: 1.2650
Epoch 14/150
782/782 - 9s - 11ms/step - accuracy: 0.6287 - loss: 1.2480


KeyboardInterrupt: 

In [22]:
loss, acc = model.evaluate(x_test, y_test, batch_size=64, verbose=2)
print(f'{acc:.2f}')

157/157 - 1s - 4ms/step - accuracy: 0.1002 - loss: 4.9081
0.10
